In [1]:
import cv2
import numpy as np

In [3]:
# Load your images
image1 = cv2.imread('XUE-Mountain-Enterance/DSC_0171.jpg')
image2 = cv2.imread('XUE-Mountain-Enterance/DSC_0172.jpg')
image3 = cv2.imread('XUE-Mountain-Enterance/DSC_0173.jpg')
image4 = cv2.imread('XUE-Mountain-Enterance/DSC_0174.jpg')
image5 = cv2.imread('XUE-Mountain-Enterance/DSC_0175.jpg')
image6 = cv2.imread('XUE-Mountain-Enterance/DSC_0176.jpg')
image7 = cv2.imread('XUE-Mountain-Enterance/DSC_0177.jpg')
image8 = cv2.imread('XUE-Mountain-Enterance/DSC_0178.jpg')
image9 = cv2.imread('XUE-Mountain-Enterance/DSC_0179.jpg')
image10 = cv2.imread('XUE-Mountain-Enterance/DSC_0180.jpg')
image11 = cv2.imread('XUE-Mountain-Enterance/DSC_0182.jpg')
image12 = cv2.imread('XUE-Mountain-Enterance/DSC_0183.jpg')
image13 = cv2.imread('XUE-Mountain-Enterance/DSC_0184.jpg')
image14 = cv2.imread('XUE-Mountain-Enterance/DSC_0185.jpg')
image15 = cv2.imread('XUE-Mountain-Enterance/DSC_0186.jpg')
image16 = cv2.imread('XUE-Mountain-Enterance/DSC_0187.jpg')

# Resize the images to the same dimensions (optional)
width, height = 800, 600  # Adjust the size as needed
image1 = cv2.resize(image1, (width, height))
image2 = cv2.resize(image2, (width, height))
image3 = cv2.resize(image3, (width, height))
image4 = cv2.resize(image4, (width, height))
image5 = cv2.resize(image5, (width, height))
image6 = cv2.resize(image6, (width, height))
image7 = cv2.resize(image7, (width, height))
image8 = cv2.resize(image8, (width, height))
image9 = cv2.resize(image9, (width, height))
image10 = cv2.resize(image10, (width, height))
image11 = cv2.resize(image11, (width, height))
image12 = cv2.resize(image12, (width, height))
image13 = cv2.resize(image13, (width, height))
image14 = cv2.resize(image14, (width, height))
image15 = cv2.resize(image15, (width, height))
image16 = cv2.resize(image16, (width, height))



In [4]:
# Initialize SIFT
sift = cv2.SIFT_create()

# Create a list to store all key points and descriptors
keypoints_list = []
descriptors_list = []

# Iterate through all images and compute key points and descriptors
images = [image1, image2, image3, image4, image5, image6, image7, image8, image9, image10, image11, image12, image13, image14, image15, image16, image17]

for img in images:
    keypoints, descriptors = sift.detectAndCompute(img, None)
    keypoints_list.append(keypoints)
    descriptors_list.append(descriptors)


In [5]:
# Create a BFMatcher (Brute Force Matcher) object
bf = cv2.BFMatcher()

# Match descriptors of the first two images
matches = bf.knnMatch(descriptors_list[0], descriptors_list[1], k=2)

# Apply ratio test to get good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Sort matches based on their distances
good_matches = sorted(good_matches, key=lambda x: x.distance)

# Draw the matches (optional)
matching_result = cv2.drawMatches(image1, keypoints_list[0], image2, keypoints_list[1], good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


In [6]:
# Extract the matching points from the good matches
src_pts = np.float32([keypoints_list[0][m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints_list[1][m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# Find the homography matrix
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)


In [7]:
# Warp image2 to align with image1
aligned_image2 = cv2.warpPerspective(image2, H, (image1.shape[1], image1.shape[0]))


In [8]:
# Create a mask for the overlapping region
mask = np.zeros_like(image1)
cv2.fillPoly(mask, [np.int32(dst_pts)], (255, 255, 255))

# Blend the two images together
result = cv2.addWeighted(image1, 0.5, aligned_image2, 0.5, 0)

# Place aligned_image2 onto the result using the mask
result = cv2.bitwise_and(result, mask)
result += aligned_image2



In [9]:
# Initialize the result as the first image
result = image1

# Iterate through the remaining images
for i in range(2, len(images) + 1):  # Start from the third image
    # Calculate homography
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # Warp the current image to align with the result
    aligned_image = cv2.warpPerspective(images[i - 1], H, (result.shape[1], result.shape[0]))
    
    # Create a mask for the overlapping region
    mask = np.zeros_like(result)
    cv2.fillPoly(mask, [np.int32(dst_pts)], (255, 255, 255))
    
    # Blend the aligned image with the result
    result = cv2.addWeighted(result, 0.5, aligned_image, 0.5, 0)
    
    # Place the aligned image onto the result using the mask
    result = cv2.bitwise_and(result, mask)
    result += aligned_image



In [10]:
# Specify the file path where you want to save the panorama
output_path = 'panorama.jpg'  # Adjust the filename and path as needed

# Save the final panorama image
cv2.imwrite(output_path, result)

print("Panorama saved as:", output_path)


Panorama saved as: panorama.jpg
